## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Preprocessing

In [13]:
path = "./drive/My Drive/data/"
train_path = path+"Train.csv"
test_path = path+"Test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
train_df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (746,835) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (700,731,740,752,761,789,811,820,829,841,850) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,Col22,Col23,Col24,Col25,Col26,Col27,Col28,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col37,Col38,Col39,Col40,...,Col2358,Col2359,Col2360,Col2361,Col2362,Col2363,Col2364,Col2365,Col2366,Col2367,Col2368,Col2369,Col2370,Col2371,Col2372,Col2373,Col2374,Col2375,Col2376,Col2377,Col2378,Col2379,Col2380,Col2381,Col2382,Col2383,Col2384,Col2385,Col2386,Col2387,Col2388,Col2389,Col2390,Col2391,Col2392,Col2393,Col2394,Col2395,Col2396,Col2397
0,RIGHGSN,1,1.086711e+04,8.648345e+03,1.576618e+04,2.890466e+04,5.273655e+03,2.346153e+04,1.285597e+04,2.871509e+04,9.114586e+03,3.043078e+04,9.293496,8.570479,10.063117,10.271758,0.548538,0.403366,NaN,NaN,1.486524,1.728324,0.993441,1.297050,0.137398,0.115987,0.260428,1.414214,1,1,1,1,1,1,0,6,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,214.0,214.0,68.0,22.0,22.0,7.0
1,RIMHI5ZGP,0,2.584312e+05,2.179633e+05,1.866287e+05,1.866287e+05,2.154519e+05,1.923615e+05,1.726538e+05,1.666974e+05,2.013518e+05,1.859404e+05,12.462385,12.280493,12.167132,12.136876,1.167898,2.676398,NaN,NaN,0.792123,0.934556,0.893203,0.966620,0.753892,0.808295,1.414214,NaN,1,1,3,4,4,0,4,5,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.0,41.0,38.0,12.0,12.0,11.0
2,RIHOG1SZU,0,1.459355e+07,1.260060e+07,1.100315e+07,1.010315e+07,1.879882e+07,1.514035e+07,5.320778e+06,6.906324e+06,9.368338e+06,1.113330e+07,16.496090,16.749305,16.532874,16.128357,1.145182,2.203874,NaN,NaN,0.422264,0.498347,0.683582,0.735339,0.808115,0.776910,1.414214,NaN,3,3,4,12,4,0,2,4,NaN,1.0,NaN,NaN,...,1,0,1,0,1,0,0,0,0,0,2,0,1,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1,1,0,378.0,378.0,310.0,21.0,21.0,16.0
3,RIHOLMQWU,0,6.727043e+05,8.181116e+05,6.794893e+05,6.794893e+05,8.257254e+05,7.605803e+05,3.577134e+05,4.236612e+05,4.304848e+05,4.065539e+05,13.419061,13.624018,13.541837,13.429097,1.204010,0.536304,NaN,NaN,0.437243,0.521341,0.623499,0.534531,-0.125829,0.388830,-1.414214,NaN,2,2,2,24,1,0,1,4,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1,2,0,1,2,0,0,0,0,0,1,2,0,0,0,0,0,1,2,0,0,0,0,0,0,702.0,702.0,644.0,951.0,951.0,743.0
4,RIHO584ET,0,-1.218742e+06,-3.005361e+06,-1.666241e+05,-2.403574e+06,2.583183e+06,2.086546e+06,5.938358e+06,5.446532e+06,3.473444e+06,3.262168e+06,NaN,14.764533,14.551021,NaN,NaN,NaN,NaN,NaN,NaN,1.344637,NaN,1.563430,-0.113101,0.617486,1.414214,NaN,4,3,4,68,8,0,2,4,NaN,3.0,NaN,NaN,...,0,0,0,0,0,0,2,1,0,0,0,0,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1305.0,1305.0,1211.0,2205.0,2205.0,1831.0


In [25]:
test_df.sample(5)

,Col1,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,Col22,Col23,Col24,Col25,Col26,Col27,Col28,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col37,Col38,Col39,Col40,Col41,...,Col2358,Col2359,Col2360,Col2361,Col2362,Col2363,Col2364,Col2365,Col2366,Col2367,Col2368,Col2369,Col2370,Col2371,Col2372,Col2373,Col2374,Col2375,Col2376,Col2377,Col2378,Col2379,Col2380,Col2381,Col2382,Col2383,Col2384,Col2385,Col2386,Col2387,Col2388,Col2389,Col2390,Col2391,Col2392,Col2393,Col2394,Col2395,Col2396,Col2397
6401,YDGOG5NJ,5589.063333,5808.070000,6934.23000,5983.153333,3641.400000,4634.833333,2776.463522,2564.638569,2565.940175,3228.905339,8.628567,8.200124,8.441356,8.696703,0.837594,0.910778,NaN,NaN,0.478035,0.704658,0.428643,0.696661,-0.005999,-0.448646,0.529373,NaN,5,4,5,5,4,1,0,6,29.0,NaN,3.0,NaN,3.0,...,1,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,10.0,10.0,9.0,8.0,8.0,4.0
10890,O9YWG1NY,11903.473330,16834.788180,15431.88917,30863.778330,12334.667270,22613.556670,23659.606070,24356.929040,17161.263590,17535.262330,9.384586,9.420169,10.026305,10.337339,1.090909,0.238910,NaN,NaN,1.405400,1.391303,0.789175,0.775431,0.098538,0.171170,0.150765,-0.866843,1,1,1,1,1,1,0,6,NaN,NaN,NaN,NaN,NaN,...,0,0,1,3,0,0,1,0,0,1,2,1,3,2,0,0,0,0,2,2,2,1,0,1,0,0,0,0,0,0,2,2,2,2,77.0,77.0,26.0,41.0,41.0,13.0
7229,72MW51YZ,33310.576670,11329.371670,12135.82333,24271.646670,6885.733333,22104.800000,15480.249760,15237.358060,10086.530020,19685.288530,10.413630,8.837207,10.003550,10.097064,0.933548,2.774284,NaN,NaN,1.366382,1.464845,0.627784,0.890544,0.224922,0.205782,0.301978,0.732802,2,1,2,2,2,1,1,6,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,1,2,1,3,0,0,0,0,1,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0,15.0,15.0,8.0,9.0,8.0,6.0
1967,O7WR61Z5,1151.430000,970.352727,889.49000,1757.475000,600.180000,766.996667,1215.093325,1158.249233,725.424414,608.445561,7.048760,6.397230,6.642482,7.471633,1.090909,0.487167,NaN,2.30099,1.252218,1.208678,0.659042,0.793283,0.164902,0.129554,0.290597,-0.878692,3,3,3,3,2,1,0,6,5.0,NaN,NaN,NaN,5.0,...,0,1,0,2,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,38.0,38.0,11.0,18.0,17.0,6.0
1156,OIHRIO45,33202.770000,27680.205000,27723.21583,55360.410000,10873.865830,21747.731670,49650.920400,58292.430470,25755.472860,33018.203760,10.410389,9.294118,9.987265,10.921620,0.998449,0.428323,NaN,NaN,1.793734,2.368566,1.052962,1.518237,0.136291,0.095844,0.247129,-0.564632,2,2,2,2,1,1,0,5,NaN,NaN,NaN,NaN,3.0,...,0,0,1,1,0,0,0,0,0,0,0,1,3,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,100.0,100.0,28.0,24.0,24.0,12.0


1st column is ID and Second columns is Target variable

In [0]:
train_df.shape, test_df.shape

((17521, 2395), (20442, 2394))

## Feature Selection